In [4]:
import requests
import re
import json
import sqlalchemy as db
from datetime import datetime


In [1]:
url="https://public.opendatasoft.com/api/records/1.0/search/?dataset=donnees-synop-essentielles-omm&q=&rows=-1&sort=-date"
url_dep="&refine.nom_dept=Haute-Vienne"
url_date="&refine.date=2016"

In [2]:
liste_dep=["Guyane","Martinique","Guadeloupe","Essonne","Gironde","Haute-Garonne","Haute-Loire","Morbiha",
"Corse-du-Sud","Finistere","Haute-Vienne","Indre-et-Loire","Vienne","Bouches-du-Rhône","Côtes-d'Armor","Haute-Corse","Hautes-Alpes","Loire-Atlantique","Nord","Alpes-Maritimes","Ille-et-Vilaine","Pyrénées-Orientales","Marne","Seine-Maritime","Lot","Hérault","Puy-de-Dôme","Haut-Rhin","Charente-Maritime",
"Manche","Meurthe-et-Moselle","Hautes-Pyrénées","Orne","Aveyron","Côte-d'Or","Bas-Rhin","Calvados","Ariège",
"Landes","La Réunion","Var","Mayotte"]

liste_annees = ["2016","2017","2018","2019","2020","2021"]




In [25]:
engine = db.create_engine("sqlite:///donnees_sncf.db")
connection = engine.connect()

metadata = db.MetaData()
with engine.connect() as con:
    con.execute("DROP TABLE meteo")

meteo_table = db.Table("meteo", metadata,
             db.Column('nom_dep',db.String()),
             db.Column('temperature', db.String()),
             db.Column('date',db.String()),
             db.Column('heure',db.String())
             )

metadata.create_all(engine)



In [26]:
for departement in liste_dep:
    for annee in liste_annees:
        data = requests.get(url + f"&refine.nom_dept={departement}" + f"&refine.date={annee}").json()
        # print(data["records"][0]["fields"])

        with engine.connect() as con:
            for record in data['records']:
                date = record["fields"]["date"].split("T")[0]
                heure = record["fields"]["date"].split("T")[1].split("+")[0]
                nom_dep= record["fields"]["nom_dept"]
                try:
                    temperature=record["fields"]["tc"]
                except:
                    temperature="None"

                insertTable = "INSERT INTO meteo (date, heure, temperature, nom_dep) VALUES (:date, :heure, :temperature, :nom_dep)"
                params = {"date":date, "heure":heure, "temperature":temperature, "nom_dep":nom_dep}
                statement = db.text(insertTable)
                con.execute(statement,params)